# Deep Mailing - XGBoost Model - Reduzindo Dimensões

O objetivo desse notebook é demonstrar a utilizacao do XGBoost para a criacao de arvores de decisão para a predição de CUPS em mailings.

Em primeiro lugar, definimos os imports que iremos usar...

In [1]:
import xgboost
import numpy as np
import os
import sys
import logging
import gc
import pickle as pickle
import pandas as pd
import dateutil.parser as parser
import os.path
import math
from sklearn.metrics import accuracy_score
from datetime import datetime
import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_tree
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams

/home/ubuntu/git/DeepMailing/env/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Abaixo definimos os diretorios e nomes dos arquivos intermediarios.

In [2]:
log_location = "../logs/"
arquivo_df_pickled_norm = "../intermediate/df.norm.pickle"
arquivo_df_pickled_norm_train = "../intermediate/df.norm.train.pickle"
arquivo_df_pickled_norm_test = "../intermediate/df.norm.test.pickle"
arquivo_df_pickled_norm_train_x = "../intermediate/df.norm.train.x.pickle.npy"
arquivo_df_pickled_norm_train_y = "../intermediate/df.norm.train.y.pickle.npy"
arquivo_df_pickled_norm_test_x = "../intermediate/df.norm.test.x.pickle.npy"
arquivo_df_pickled_norm_test_y = "../intermediate/df.norm.test.y.pickle.npy"

Definimos o versampling que é o minimo de rows com a variavel alvo setado no dataset de treinamento

In [3]:
Oversampling = True
RateOversampling = 0.3

Redefinimos o logger que iremos usar

In [4]:
logger = logging.getLogger()
logging.basicConfig(format="%(asctime)-15s %(message)s",
                    level=logging.DEBUG,
                    filename=os.path.join(log_location,'xgboost.log.' + datetime.now().strftime("%Y%m%d%H%M%S.%f") + '.log'))

Criamos uma função para imprimir tanto no log quanto no notebook...

In [5]:
def print_log(msg):
    logging.debug(msg)
    print(msg)    

Carregamos para a memoria o arquivo normalizado e pickled que foi gerado no notebook de "Preparação de Dados".

In [6]:
print_log("Carregando Pickling normalizado:{}".format(arquivo_df_pickled_norm))    
chamadas = pd.read_pickle(arquivo_df_pickled_norm)

Carregando Pickling normalizado:../intermediate/df.norm.pickle


Verificamos as dimensões do dataframe carregado.... E imprimimos uma amostra do dado que precisamos com apenas as colunas relevantes... Como podemos perceber o nosso modelo considera apenas colunas com valores booleanos (0 ou 1)

In [7]:
chamadas.head(10)

,id,PROPENSAO,NUMERO,DATA_MAILING,NORM_CARTEIRA_W02,NORM_CARTEIRA_W01,NORM_CARTEIRA_A02,NORM_CARTEIRA_A01,NORM_CARTEIRA_A03,NORM_SEGMENTO_FA,...,NORM_DDD_77,NORM_DDD_49,NORM_DDD_74,NORM_DDD_62,NORM_DDD_37,NORM_DDD_12,NORM_DDD_89,NORM_TENTATIVAS,NORM_LIGACOES,NORM_CUP
0,1808947,NaN,-996934974.0,2017-11-28,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1808948,NaN,-996038652.0,2017-11-28,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,1,0
2,1808949,NaN,-996581839.0,2017-11-28,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1808950,ALTA PROPENSÃƒO DE PAGAMENTO PARCELADO.,-32234178.0,2017-11-28,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1808951,NaN,-32365578.0,2017-11-28,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1808952,ALTA PROPENSÃƒO DE PAGAMENTO PARCELADO.,-998764576.0,2017-11-28,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,1,0
6,1808953,NaN,-38084249.0,2017-11-28,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,1808954,ALTA PROPENSÃƒO DE PAGAMENTO PARCELADO.,-988019800.0,2017-11-28,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
8,1808955,NaN,-999254115.0,2017-11-28,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1808956,NaN,-994169573.0,2017-11-28,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [8]:
print_log(chamadas.shape)
chamadas.loc[:, 'NORM_CARTEIRA_A01':'NORM_LIGACOES'].head(10)

(9586480, 106)


,NORM_CARTEIRA_A01,NORM_CARTEIRA_A03,NORM_SEGMENTO_FA,NORM_SEGMENTO_LC,NORM_SEGMENTO_CR,NORM_SEGMENTO_FC,NORM_SEGMENTO_CC,NORM_SEGMENTO_MA,NORM_SEGMENTO_HC,NORM_SEGMENTO_FT,...,NORM_DDD_41,NORM_DDD_77,NORM_DDD_49,NORM_DDD_74,NORM_DDD_62,NORM_DDD_37,NORM_DDD_12,NORM_DDD_89,NORM_TENTATIVAS,NORM_LIGACOES
0,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,1
2,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,1,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
8,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Para tratarmos as colunas de forma correta, precisamos eliminar os caracteres especiais das colunas, principalmente o espaco e o sinal de - para isso iremos rodar o codigo abaixo para tratar.

In [9]:
cols = chamadas.columns
cols = cols.map(lambda x: x.replace(' ', '_').replace('-','menos'))
chamadas.columns = cols

Um problema que temos é q temos muito poucos CUPS sendo acionados, então temos q remover linhas não relevantes no arquivo.

As linhas que devemos remover são aquelas em que não houve CUP e cujas features apesar de ativadas (terem valor 1), raramente são determinantes para um CUP.

No exemplo abaixo, eu vou determinar que se em menos de 15% das vezes em q determinada feature foi ativada, foi ativado um CUP tambem, eu vou determinar que essa coluna é irrelevante e que todos os valores com ativação dessa coluna e que não haja CUP seja apagados.

Dessa forma, iremos remover linhas ativações de features que não geraram NORM_CUP =1 e q sabemos q não são relevantes ao modelo. 

No final, calculamos o threshold, que é o numero minimo de CUPs ativados para determinarmos se aquela feature é relevante ou não.

In [10]:
minima_expressividade = 0.15
dimensionalidade_cup = minima_expressividade / (len(chamadas.query("NORM_CUP==1").index) /  len(chamadas.index))
dimensionalidade_cup 

60.94134599084591

Como já calculamos o numero minimo de CUPs gerados para que uma determinada feature seja gerada, podemos agora determinar quais as colunas que devem ser limpas do dataframe 

In [11]:
def limpar_coluna_df(dataframe, coluna):
    return dataframe.query("not ({} == 1 and NORM_CUP==0)".format(coluna))

def coluna_tem_cup(dataframe, coluna):
    return len(dataframe.query("{} == 1 and NORM_CUP==1".format(coluna)).index) > dimensionalidade_cup
    
colunas_para_limpar = []
lista_colunas = chamadas.loc[:, 'NORM_CARTEIRA_A01':].columns.values
for coluna in list(lista_colunas):
    if not coluna_tem_cup(chamadas,coluna):
        colunas_para_limpar.append(coluna)
        print_log("Coluna {} foi limpa do dataframe por ser irrelevante...".format(coluna))
        


Coluna NORM_SEGMENTO_FT foi limpa do dataframe por ser irrelevante...
Coluna NORM_PROPENSAO_ALTA foi limpa do dataframe por ser irrelevante...
Coluna NORM_STATUS_INTERNA_Novo foi limpa do dataframe por ser irrelevante...
Coluna NORM_STATUS_TELEFONE_Sem_Tel foi limpa do dataframe por ser irrelevante...
Coluna NORM_STATUS_TELEFONE_Novo foi limpa do dataframe por ser irrelevante...
Coluna NORM_DDD_96 foi limpa do dataframe por ser irrelevante...
Coluna NORM_DDD_97 foi limpa do dataframe por ser irrelevante...
Coluna NORM_DDD_99 foi limpa do dataframe por ser irrelevante...
Coluna NORM_DDD_63 foi limpa do dataframe por ser irrelevante...
Coluna NORM_DDD_92 foi limpa do dataframe por ser irrelevante...
Coluna NORM_DDD_28 foi limpa do dataframe por ser irrelevante...
Coluna NORM_DDD_68 foi limpa do dataframe por ser irrelevante...
Coluna NORM_DDD_93 foi limpa do dataframe por ser irrelevante...
Coluna NORM_DDD_46 foi limpa do dataframe por ser irrelevante...
Coluna NORM_DDD_95 foi limpa do d

Tendo as colunas irrelevantes, podemos agora limpar elas do nosso dataframe, oque vai fazer com ele fique com menos tamanho e com melhor precisão.

In [12]:
for coluna in colunas_para_limpar:
    chamadas = limpar_coluna_df(chamadas,coluna)    
    gc.collect()

Podemos agora analisar o tamanho final do nosso dataframe:

In [13]:
total_chamadas = len(chamadas.index)
print(total_chamadas)

8944714


Perfeito, então vamos embaralhar os dados e gerar os nossos dados de treinamento e teste. Vamos considerar 70% para treinamento e 30 % para teste. No final, apagamos o dataframe lido para economizar memoria

In [14]:
print_log("Criando Pickling de train e teste...")
chamadas = chamadas.sample(int(len(chamadas.index)))
chamadas_train = chamadas.tail(int(len(chamadas.index) * 0.7))
chamadas_test = chamadas.head(int(len(chamadas.index) * 0.3))
del chamadas

Criando Pickling de train e teste...


In [15]:
if Oversampling:
    print_log("Devemos Fazer oversampling...")
    chamadas_train_count = len(chamadas_train.index)
    chamadas_train_cup = chamadas_train.query("NORM_CUP == 1")
    chamadas_train_cup_count = len(chamadas_train_cup.index)
    multiplicador = (RateOversampling / (chamadas_train_cup_count / chamadas_train_count))
    print_log("RateOversampling:{}".format(RateOversampling))
    print_log("chamadas_train_cup:{}".format(chamadas_train_cup_count))
    print_log("chamadas_train_count:{}".format(chamadas_train_count))
    print_log("multiplicador:{}".format(multiplicador))
    chamadas_train = chamadas_train.append([chamadas_train_cup] * int(multiplicador), ignore_index = True )
    

Devemos Fazer oversampling...
RateOversampling:0.3
chamadas_train_cup:16425
chamadas_train_count:6261299
multiplicador:114.36162557077625


Criamos uma função para gerar um arquivo de referencia de colunas a serem usadas q que vai ser importante na hora de gerar a arvore de decisao...

In [16]:
def create_column_reference(header_chamadas_x,arquivo_df_pickled_norm_train_x):
    print_log("Criando Arquivo de referencia de colunas...")
    with open(arquivo_df_pickled_norm_train_x+".txt","w") as f:
        counter = 0
        lista_header = list(header_chamadas_x.columns.values)
        for header in lista_header:
            f.write("{}-{}\n".format(counter,header))
            counter=counter+1

Criamos agora os nossos dataframes de X que são as features e as Y que são os alvos de predição. Também removemos qualquer linha em tentativas seja igual a zero, além de criar um arquivo de referencia com as colunas X que serão usadas no modelo. Esses dataframes serão convertidos para matrizes no formato numpy

In [17]:
print_log("Separando colunas em X e Y...")        
create_column_reference(chamadas_train.loc[:, 'NORM_CARTEIRA_A01':'NORM_LIGACOES'].head(1), arquivo_df_pickled_norm_train_x)
chamadas_train_x = chamadas_train.loc[:, 'NORM_CARTEIRA_A01':'NORM_LIGACOES'].as_matrix()
chamadas_train_y = chamadas_train.NORM_CUP.as_matrix()
chamadas_test_x = chamadas_test.loc[:, 'NORM_CARTEIRA_A01':'NORM_LIGACOES'].as_matrix()
chamadas_test_y = chamadas_test.NORM_CUP.as_matrix()


Separando colunas em X e Y...
Criando Arquivo de referencia de colunas...


Após a criação das matrizes numpy, gravamos elas em arquivos.

In [18]:
print_log("Criando arquivos finais em formato NUMPY para consumo pelo algoritmo...")        
np.save(arquivo_df_pickled_norm_train_x,chamadas_train_x)
np.save(arquivo_df_pickled_norm_train_y,chamadas_train_y)
np.save(arquivo_df_pickled_norm_test_x,chamadas_test_x)
np.save(arquivo_df_pickled_norm_test_y,chamadas_test_y)


Criando arquivos finais em formato NUMPY para consumo pelo algoritmo...


Apagamos todos os dados intermediários e rodamos o garbage collector para economizar memória.

In [19]:
print_log("Removendo objetos desnecessarios")        
del chamadas_train_x
del chamadas_train_y
del chamadas_train
del chamadas_test
del chamadas_test_x
del chamadas_test_y
gc.collect()

Removendo objetos desnecessarios


1245

Carregamos os objetos numpy em memoria

In [20]:
print_log("Carregando objetos numpy")        
train_x = np.load(arquivo_df_pickled_norm_train_x)
train_y = np.load(arquivo_df_pickled_norm_train_y)
test_x = np.load(arquivo_df_pickled_norm_test_x)
test_y = np.load(arquivo_df_pickled_norm_test_y)

Carregando objetos numpy


Contamos quantos CUPS existem em treinamento e teste...

In [21]:
msg1 = "Train - CUPS Detectados {} num universo de {}".format(len([y for y in train_y if y >0]),len(train_y))
msg2 = "Test - CUPS Detectados {} num universo de {}".format(len([y for y in test_y if y >0]),len(test_y))
print_log(msg1)
print_log(msg2)

Train - CUPS Detectados 1888875 num universo de 8133749
Test - CUPS Detectados 7171 num universo de 2683414


Configuramos os parametros para o XGBoost, especificando que queremos que ele seja o mais exato possivel, que a medida de erro é erro simples e que queremos apenas uma classificacao binaria com um maximo de 1000 interações

In [ ]:
param = {}
param['eta'] = 0.2
param['objective'] = 'binary:logistic'
param['eval_metric'] = 'error'
param['tree_method'] = 'exact'
param['silent'] = 0
num_round = 1000

Após a definicão dos paramêtros de teste, criamos as matrizes no formato do XGBoost e treinamos o modelo.

In [ ]:
gc.collect()
print_log("Starting model for params:{}".format(param))
dtrain = xgb.DMatrix(train_x, train_y)
dtest = xgb.DMatrix(test_x, test_y)
gpu_res = {}
booster = xgb.train(param, dtrain, num_round, evals=[], evals_result=gpu_res)

Starting model for params:{'tree_method': 'exact', 'eval_metric': 'error', 'silent': 0, 'eta': 0.2, 'objective': 'binary:logistic'}


Após o modelo ser treinado, podemos plotar ele... Para verificar que coluna é cada feature no modelo, por favor ver a lista em anexo no final desse notebook.

In [ ]:
%matplotlib inline
rcParams['figure.figsize'] = 80,50
plot_tree(booster, rankdir='LR')
plt.show()

Agora, vamos tentar predizer os dados com o nosso modelo treinado...

In [ ]:
test_y_pred = booster.predict(dtest)
test_predictions = np.array([value for value in test_y_pred])

E Finalmente medir a precisão da nossa predição... Tanto no total quanto em CUPs detectados.

In [ ]:
accuracy = accuracy_score(test_y, test_predictions.round())
print_log("CUPS Previstos:{}".format(len([x for x in test_predictions if x > 0.5])))
print_log("CUPS na Base Teste:{}".format(len([x for x in test_y if x > 0.5])))
print_log("Accuracy Total:{}".format(accuracy))
print_log("Accuracy em CUPs:{}".format(len([x for x in test_predictions if x > 0.5]) / len([x for x in test_y if x > 0.5])))

Após, vamos salvar o modelo gerado em um arquivo para reuso...

In [ ]:
save_file = "../output/{}.model".format(datetime.now().strftime("%Y%m%d.%H%M%S"))
with open(save_file, 'wb') as fp:
    pickle.dump(booster, fp)    
print_log("Model saved as {}".format(save_file))

In [ ]:
%%bash 

cat ../intermediate/df.norm.train.x.pickle.npy.txt  